In [1]:
from lxml.html import fromstring

import pandas as pd
import numpy as np

import requests
import time
import os

# 데뷔년도 및 생년월일 붙이기 (from KBO)

In [2]:
pitcher = pd.read_csv('선수데이터(전처리완료)/투수.csv')
hitter = pd.read_csv('선수데이터(전처리완료)/타자.csv')


hitter_info = hitter[['ID', '선수명']].drop_duplicates().reset_index(drop=True)
pitcher_info = pitcher[['ID', '선수명']].drop_duplicates().reset_index(drop=True)

In [3]:
hitter_start_years = []
hitter_birthdays = []
pitcher_start_years = []
pitcher_birthdays = []

# 타자
for ID in hitter_info['ID']:
    url = 'https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId={}'.format(ID)
    res = requests.get(url)
    parser = fromstring(res.text)
    
    try:
        start_year = parser.xpath('//*[@id="contents"]/div[2]/div[1]/div[1]/ul/li[9]')[0].text_content().split(': ')[1][:2]
    except:
        print(f'{ID} Error')
    
    try:
        birthday = parser.xpath('//*[@id="contents"]/div[2]/div[1]/div[1]/ul/li[3]')[0].text_content().split(': ')[1]
    except:
        print(f'{ID} Error')
        
        
    hitter_start_years.append(start_year)
    hitter_birthdays.append(birthday)
    

# 투수
for ID in pitcher_info['ID']:
    url = 'https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId={}'.format(ID)
    res = requests.get(url)
    parser = fromstring(res.text)
    
    try:
        start_year = parser.xpath('//*[@id="contents"]/div[2]/div[1]/div[1]/ul/li[9]')[0].text_content().split(': ')[1][:2]
    except:
        print(f'{ID} Error')
        
    try:
        birthday = parser.xpath('//*[@id="contents"]/div[2]/div[1]/div[1]/ul/li[3]')[0].text_content().split(': ')[1]
    except:
        print(f'{ID} Error')
        
    pitcher_start_years.append(start_year)
    pitcher_birthdays.append(birthday)
    
    
hitter_info['데뷔년도'] = hitter_start_years
hitter_info['생년월일'] = hitter_birthdays
pitcher_info['데뷔년도'] = pitcher_start_years
pitcher_info['생년월일'] = pitcher_birthdays

In [4]:
# 긁어온 데이터 간단한 전처리
def year_preprocessing(x):

    if x in ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']:
        x = '20'+x

    elif x in ['86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']:
        x = '19'+x
        
    return x


def birth_preprocessing(x):
    
    x = x.replace('년 ', '-').replace('월 ', '-').replace('일','')
    
    return x

hitter_info['데뷔년도'] = hitter_info['데뷔년도'].apply(lambda x: year_preprocessing(x))
pitcher_info['데뷔년도']= pitcher_info['데뷔년도'].apply(lambda x: year_preprocessing(x))
hitter_info['생년월일'] = hitter_info['생년월일'].apply(lambda x: birth_preprocessing(x))
pitcher_info['생년월일'] = pitcher_info['생년월일'].apply(lambda x: birth_preprocessing(x))

In [5]:
hitter_final = pd.merge(hitter_info, hitter, on=['ID', '선수명'], how='right')
pitcher_final = pd.merge(pitcher_info, pitcher, on=['ID', '선수명'], how='right')

hitter_final.to_csv('선수데이터(전처리완료)/타자.csv', index=False, encoding='utf-8-sig')
pitcher_final.to_csv('선수데이터(전처리완료)/투수.csv', index=False, encoding='utf-8-sig')